### SQL Alchemy

In [1]:
import sqlalchemy

from sqlalchemy import create_engine, Column, Integer, Float, String
from sqlalchemy.orm import declarative_base

In [2]:
# Create an SQLite in-memory database engine
Engine = create_engine("sqlite:///:memory:", echo = True)

In [3]:
# Create a base class for declarative class definitions
Base = declarative_base()

In [4]:
# Define the Animal class, representing the 'animal' table in the database
class Animal(Base):

    # Define table name
    __tablename__ = 'animal'

    # Define columns for the table
    aid = Column(Integer, primary_key = True)
    aname = Column(String, nullable = False)
    acategory = Column(String(18))
    timetofeed = Column(Float)

    # Define the __repr__ method to return a readable string representation of each Animal instance
    def __repr__(self):
        
        return "<Animal(AID = %d, AName = '%s', ACategory = '%s', TimeToFeed = '%s')>" % (self.aid, self.aname, self.acategory, self.timetofeed)

In [5]:
# Generate the 'animal' table schema in the database
Animal.__table__

Table('animal', MetaData(), Column('aid', Integer(), table=<animal>, primary_key=True, nullable=False), Column('aname', String(), table=<animal>, nullable=False), Column('acategory', String(length=18), table=<animal>), Column('timetofeed', Float(), table=<animal>), schema=None)

In [6]:
# Create all tables defined in the base metadata
Base.metadata.create_all(Engine)

2024-11-08 12:16:23,031 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-08 12:16:23,034 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("animal")
2024-11-08 12:16:23,035 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-11-08 12:16:23,038 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("animal")
2024-11-08 12:16:23,039 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-11-08 12:16:23,041 INFO sqlalchemy.engine.Engine 
CREATE TABLE animal (
	aid INTEGER NOT NULL, 
	aname VARCHAR NOT NULL, 
	acategory VARCHAR(18), 
	timetofeed FLOAT, 
	PRIMARY KEY (aid)
)


2024-11-08 12:16:23,042 INFO sqlalchemy.engine.Engine [no key 0.00122s] ()
2024-11-08 12:16:23,044 INFO sqlalchemy.engine.Engine COMMIT


---
### Load Data with SQL Alchemy

In [7]:
from sqlalchemy.orm import sessionmaker

In [8]:
# Create instances of the Animal class
animal1 = Animal(aid = 1, aname = 'Galapagos Penguin', acategory = 'exotic', timetofeed = '0.5')
animal2 = Animal(aid = 2, aname = 'Emperor Penguin', acategory = 'rare', timetofeed = '0.75')
animal3 = Animal(aid = 3, aname = 'Sri Lankan sloth bear', acategory = 'exotic', timetofeed = '2.5')
animal4 = Animal(aid = 4, aname = 'Grizzly Bear', acategory = 'common', timetofeed = '3.0')

In [9]:
# Access an attribute to retrieve the name of a specific animal
animal1.aname

'Galapagos Penguin'

In [10]:
# Access an attribute to retrieve the category of a specific animal
animal3.acategory

'exotic'

In [11]:
# Create a session factory bound to the engine, allowing interaction with the database
Session = sessionmaker(bind = Engine)
session = Session() # Instantiate a session object to manage transactions with the database

In [12]:
# Add individual instances to the session
session.add(animal1)
session.add(animal2)
session.add_all([animal3, animal4])

In [13]:
# Modify the 'aname' attribute of animal3
animal3.aname = 'Sri Lankan Sloth Bear'

In [14]:
# Commit the transaction to the database
session.commit()

2024-11-08 12:16:23,212 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-08 12:16:23,217 INFO sqlalchemy.engine.Engine INSERT INTO animal (aid, aname, acategory, timetofeed) VALUES (?, ?, ?, ?)
2024-11-08 12:16:23,219 INFO sqlalchemy.engine.Engine [generated in 0.00223s] [(1, 'Galapagos Penguin', 'exotic', 0.5), (2, 'Emperor Penguin', 'rare', 0.75), (3, 'Sri Lankan Sloth Bear', 'exotic', 2.5), (4, 'Grizzly Bear', 'common', 3.0)]
2024-11-08 12:16:23,223 INFO sqlalchemy.engine.Engine COMMIT


---
### Queries in SQL Alchemy

In [15]:
# Query the 'Animal' table to retrieve the first entry where 'acategory' is 'rare'
session.query(Animal).filter_by(acategory = 'rare').first()

2024-11-08 12:16:23,243 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-08 12:16:23,249 INFO sqlalchemy.engine.Engine SELECT animal.aid AS animal_aid, animal.aname AS animal_aname, animal.acategory AS animal_acategory, animal.timetofeed AS animal_timetofeed 
FROM animal 
WHERE animal.acategory = ?
 LIMIT ? OFFSET ?
2024-11-08 12:16:23,251 INFO sqlalchemy.engine.Engine [generated in 0.00226s] ('rare', 1, 0)


<Animal(AID = 2, AName = 'Emperor Penguin', ACategory = 'rare', TimeToFeed = '0.75')>

In [16]:
# Query the 'Animal' table to retrieve all entries where 'acategory' is 'exotic'
session.query(Animal).filter_by(acategory = 'exotic').all()

2024-11-08 12:16:23,276 INFO sqlalchemy.engine.Engine SELECT animal.aid AS animal_aid, animal.aname AS animal_aname, animal.acategory AS animal_acategory, animal.timetofeed AS animal_timetofeed 
FROM animal 
WHERE animal.acategory = ?
2024-11-08 12:16:23,278 INFO sqlalchemy.engine.Engine [generated in 0.00130s] ('exotic',)


[<Animal(AID = 1, AName = 'Galapagos Penguin', ACategory = 'exotic', TimeToFeed = '0.5')>,
 <Animal(AID = 3, AName = 'Sri Lankan Sloth Bear', ACategory = 'exotic', TimeToFeed = '2.5')>]

In [17]:
# Query the 'Animal' table to retrieve 'acategory' and 'aname' columns
for (acategory, aname) in session.query(Animal.acategory, Animal.aname):
    print(acategory, ' ', aname)

2024-11-08 12:16:23,312 INFO sqlalchemy.engine.Engine SELECT animal.acategory AS animal_acategory, animal.aname AS animal_aname 
FROM animal
2024-11-08 12:16:23,313 INFO sqlalchemy.engine.Engine [generated in 0.00169s] ()
exotic   Galapagos Penguin
rare   Emperor Penguin
exotic   Sri Lankan Sloth Bear
common   Grizzly Bear


In [18]:
# Query the 'Animal' table and ordering the results by 'aname'
for instance in session.query(Animal).order_by(Animal.aname):
    print(instance.aid, instance.aname, instance.timetofeed)

2024-11-08 12:16:23,339 INFO sqlalchemy.engine.Engine SELECT animal.aid AS animal_aid, animal.aname AS animal_aname, animal.acategory AS animal_acategory, animal.timetofeed AS animal_timetofeed 
FROM animal ORDER BY animal.aname
2024-11-08 12:16:23,340 INFO sqlalchemy.engine.Engine [generated in 0.00123s] ()
2 Emperor Penguin 0.75
1 Galapagos Penguin 0.5
4 Grizzly Bear 3.0
3 Sri Lankan Sloth Bear 2.5


In [19]:
# Query the 'Animal' table to retrieve all entries with 'acategory' set to 'exotic' and ordering the results by 'aname'
for instance in session.query(Animal).filter_by(acategory = 'exotic').order_by(Animal.aname).all():
    print(instance)

2024-11-08 12:16:23,368 INFO sqlalchemy.engine.Engine SELECT animal.aid AS animal_aid, animal.aname AS animal_aname, animal.acategory AS animal_acategory, animal.timetofeed AS animal_timetofeed 
FROM animal 
WHERE animal.acategory = ? ORDER BY animal.aname
2024-11-08 12:16:23,370 INFO sqlalchemy.engine.Engine [generated in 0.00204s] ('exotic',)
<Animal(AID = 1, AName = 'Galapagos Penguin', ACategory = 'exotic', TimeToFeed = '0.5')>
<Animal(AID = 3, AName = 'Sri Lankan Sloth Bear', ACategory = 'exotic', TimeToFeed = '2.5')>
